In [ ]:
# Import necessary libraries
import os
import torch
import pandas as pd
from torch import nn
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from transformers import BertModel, BertTokenizer, CamembertModel, CamembertTokenizer, AdamW, get_linear_schedule_with_warmup

# Import custom modules
import load_data
from training_audio_model import *
from utils import *

# Set device to GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())

# Specify the path to the transcription data
transcr_path = 'paco-cheese/transcr'

# Load the data using the custom load_data module
data = load_data.load_all_ipus(folder_path=transcr_path, load_words=True)

# Display the first 20 rows of the data
data[:20]

In [ ]:
# Create the target variable y using the custom function create_y
y = create_y(data)

# Calculate the size of the training set as 80% of the total data
train_size = int(len(data) * 0.8)

# Split the data into training and test sets
X_train = data[:train_size]
y_train = y[:train_size]
X_test = data[train_size:]
y_test = y[train_size:]

# Display the first 20 elements of the test target variable
y_test[:20]

In [ ]:
# Define a function to predict the target variable y
def pred_y(df, x):
    y = []
    # Reset the index of the dataframe to avoid KeyError
    df = df.reset_index(drop=True)
    for i in range(len(df)):
        # Check if the index i+1 is within the range of the dataframe
        if i + 1 in df.index:
            # Calculate the waiting time until the next speech starts
            wait_time = df.loc[i + 1, 'start_words'] - df.loc[i, 'stop_words']
            # Check if the waiting time is longer than x seconds and if the speaker changes
            if wait_time > x:
                y.append(1)  # There is a change of speech and the waiting time is longer than x seconds
            else:
                y.append(0)  # No significant speaker change or no speaker change
        else:
            y.append(0)  # No next speaker to compare, so append 0
    return y

# Predict the target variable y for the test set using the custom function pred_y
y_pred = pred_y(X_test, 0.9)

In [ ]:
# Calculate the F1 score for the test set
f1 = f1_score(y_test,y_pred, average='weighted')

# Calculate the confusion matrix for the test set
conf_matrix = confusion_matrix(y_test, y_pred)

# Print the F1 score and confusion matrix
print(f'Test F1 Score: {f1}')
print(f'Confusion Matrix:\n{conf_matrix}')

# Calculate the total number of instances for each class
total_class_0 = np.sum(conf_matrix[0])
total_class_1 = np.sum(conf_matrix[1])

# Calculate the number of correctly detected instances for each class
detected_class_0 = conf_matrix[0, 0]  # True positives for class 0
detected_class_1 = conf_matrix[1, 1]  # True positives for class 1

# Print the number of correctly detected instances for each class
print(f'Number of class 0 instances detected: {detected_class_0} out of {total_class_0}')
print(f'Number of class 1 instances detected: {detected_class_1} out of {total_class_1}')